<a href="https://colab.research.google.com/github/dineshnain00/CODING-SAMURAI-INTERNSHIP-TASK-/blob/main/Project%206%3A%20Sentiment_Analysis_on_Social_Media_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
#— loads Python's regular expression module (used to find and replace text patterns).

In [3]:
import pandas as pd
#— loads pandas (a table/dataframe library) and calls it pd for short.

In [4]:
import nltk
#— loads NLTK (Natural Language Toolkit), a common NLP utility library.

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#— imports VADER, a sentiment tool tuned for short social-media text.

In [6]:
from textblob import TextBlob
#— imports TextBlob, a simple library that gives a polarity score for text.

In [7]:
nltk.download('vader_lexicon')
#— makes sure the VADER word-list is available on your machine (downloads it the first time).

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [9]:
def clean_tweet(text):
   # """
   # Basic tweet/text cleaning:
   # - convert to string & lowercase
    #- remove urls, mentions, 'RT', hashes (# symbol), and non-alphanumeric chars
  #  - collapse extra spaces
   # """
    text = str(text)
    text = text.lower()
    text = re.sub(r'http\S+|www\.\S+', '', text)        # remove URLs
    text = re.sub(r'@\w+', '', text)                  # remove mentions
    text = re.sub(r'#', '', text)                     # remove the hash symbol but keep the word
    text = re.sub(r'\brt\b', '', text)                # remove standalone RT
    text = re.sub(r'[^a-z0-9\s]', '', text)           # remove punctuation & emojis (simple approach)
    text = re.sub(r'\s+', ' ', text).strip()          # collapse whitespace and trim
    return text

In [10]:
def vader_sentiment(text, analyzer):
    scores = analyzer.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        label = 'positive'
    elif compound <= -0.05:
        label = 'negative'
    else:
        label = 'neutral'
    return compound, label

def textblob_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0.05:
        label = 'positive'
    elif polarity < -0.05:
        label = 'negative'
    else:
        label = 'neutral'
    return polarity, label

In [17]:
df=pd.read_csv("CyberBullying Comments Dataset.csv")
print(df)
#Option B (if you don't have a CSV): use a small sample list
tweets = ["I love this product!", "Worst service ever :(", "I'm not sure..."]
df = pd.DataFrame({'text': tweets})

                                                    Text  CB_Label
0      damn there is someones nana up here at beach w...         0
1      no kidding! dick clark was a corpse mechanical...         0
2      i read an article on jobros and thought damn w...         0
3      I got one fucking day of sprinkles and now it'...         0
4      I was already listening to Elliott smith  and ...         0
...                                                  ...       ...
11095  "Don't worry you little empty head over it ......         1
11096  "Some of Ya'll are dumb as fuck.... These are ...         1
11097  "Lana, you're so full of shit your eyes are br...         1
11098  "You ain't lying let the @dbeeio61:disqus\xa0\...         1
11099  "Looks like that little Cut-n-paste job has go...         1

[11100 rows x 2 columns]


In [18]:
#Preprocess text
df['clean_text'] = df['text'].apply(clean_tweet)

# Initialize the VADER analyzer
analyzer = SentimentIntensityAnalyzer()

# Apply sentiment functions and expand results into columns
df[['vader_compound', 'vader_label']] = df['clean_text'].apply(
    lambda t: pd.Series(vader_sentiment(t, analyzer))
)
df[['tb_polarity', 'tb_label']] = df['clean_text'].apply(
    lambda t: pd.Series(textblob_sentiment(t))
)

In [19]:
#Simple ensemble / final label: if both agree use that, otherwise prefer VADER
def combined_label(row):
    if row['vader_label'] == row['tb_label']:
        return row['vader_label']
    return row['vader_label']

df['label'] = df.apply(combined_label, axis=1)

In [20]:
# Summary statistics
summary_counts = df['label'].value_counts()
summary_pct = df['label'].value_counts(normalize=True).mul(100).round(2)

print("Counts:\n", summary_counts)
print("\nPercentages:\n", summary_pct)

# Save results to CSV
df.to_csv('tweets_with_sentiment.csv', index=False)

Counts:
 label
negative    2
positive    1
Name: count, dtype: int64

Percentages:
 label
negative    66.67
positive    33.33
Name: proportion, dtype: float64
